### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 3 - Digital Video Processing
In this assignment, you will use OpenCV and FFmpeg to implement very basic video editing functions. These tasks include:

1. Create a slide show (as a video) from images, and optionally create the slideshow as greyscale video.
2. Extract the audio track from a video file.
3. Replace the audio track in a video file.
4. Combine two or more videos into one video file.
5. Blend an image (fade-in/fade-out) with a video.
6. Blend two videos into one video (video collage).

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.

## Setup

For general installation instructions, please refer to the ressources given for all the assignments [in Moodle](https://moodle.univie.ac.at/course/view.php?id=164140#section-12).

If the cell below executes without error, you can start the assignment!

In [10]:
# -------- Imports --------
# Please do not change the contents of this cell!

# Imports required by us.
import cv2              # opencv-python
import ffmpeg           # ffmpeg-python
import subprocess   # for calling local executables such as ffmpeg.exe
import pandas as pd  # pandas
from fractions import Fraction as frac # simplifying fractions

In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [11]:
# Place your own imports here.

#This imports were missing from the top cell
import IPython
from IPython.core.display import HTML

import math
import os
import glob

from enum import Enum

In [12]:
# Place any helper functions, global variables and classes here.

# For example:the function you need to play back a mp4-video file.
# You may use this function to display the videos in your definition file during the assessment for demoing the solutions.
def VideoPlayer(videoFile: str) -> None:
    
    IPython.display.display(
        HTML("""
            <video alt="test" controls>
            <source src={} type="video/mp4">
            </video>
        """.format(videoFile))
    )
    
    return

def outputDir(dir: str):
    if not os.path.exists(dir):
        os.mkdir(dir)

In [41]:
#VideoPlayer("results/t6/output.mp4")


## Task 3.1: Create a slide show (Video) from multiple images and convert it to greyscale 

In [4]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.
def CreateVideoFromImages(
        inImgLibFolder: str,   \
        imageFormat: str,       \
        durationInSec: float,    \
        convertToGreyScale: bool, \
        outFolder: str,            \
        outVideo: str) -> None:
    
    outputDir(outFolder)#Generate the output folder if it doesn't exist

    #                             Image folder               REGEX MOTOR          DURATION PER IMAGE
    inp = ffmpeg.input(inImgLibFolder+'/*.'+imageFormat, pattern_type='glob', framerate=1/durationInSec)

    if convertToGreyScale:
        inp = inp.filter("format", "gray")
    
    inp.output(outFolder+'/'+outVideo+'.mp4')\
        .run(overwrite_output=True)

In [8]:
# Test your function here.
#CreateVideoFromImages("resources/images", "JPG", 2, True, "results/t1", "output")

## Task 3.2: Extract the Audio Track from a Video File 

In [9]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.
#Adapted from https://www.reddit.com/r/ffmpeg/comments/y4xtzs/using_ffmpeg_to_split_the_audio_and_video_into/
def SplitAudioVideoTracks(
        inVideo: str,     \
        outFolder: str,    \
        outVideoTrack: str, \
        outAudioTrack: str) -> None:
    
    outputDir(outFolder)#Generate the output folder if it doesn't exist
    
    inp = ffmpeg.input(inVideo)
    
    if outVideoTrack != None:
        #                 OUTPUT DIR                       VIDEO CODED    DISABLE AUDIO
        inp.output(outFolder+"/"+outVideoTrack+".mp4", **{"c:v": "copy"}, an=None)\
        .run(overwrite_output=True)
    
    if outAudioTrack != None:
        #                 OUTPUT DIR                       AUDIO CODED    DISABLE VIDEO
        inp.output(outFolder+"/"+outAudioTrack+".mp4", **{"c:a": "copy"}, vn=None)\
        .run(overwrite_output=True) #ffmpeg -i infile.mp4 -an -c:v copy videoout.mp4 -vn -c:a copy audioout.mp4


In [10]:
# Test your function here.
#SplitAudioVideoTracks("resources/video/sample1.mp4", "results/t2", "videoOnly", "audioOnly")

## Task 3.3: Replace the Audio Track in a Video File

In [13]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.

def AddOrReplaceAudio(
        inVideo: str, \
        inAudio: str,  \
        outFolder:str,  \
        outVideo: str) -> None:
    
    outputDir(outFolder)#Generate the output folder if it doesn't exist
    
    if inAudio == None: #If no audio is provided, the video is muted
        SplitAudioVideoTracks(inVideo, outFolder, outVideo, None)
        return
    
    #JOIN STREAMS   VIDEO STREAM              AUDIO STREAM      AUDIO TRUE  VIDEO TRUE
    ffmpeg.concat(ffmpeg.input(inVideo), ffmpeg.input(inAudio),     a=1    ,    v=1)\
    .output(outFolder+"/"+outVideo+".mp4", t=float(ffmpeg.probe(inVideo)["streams"][0]["duration"])).run(overwrite_output=True)
    #                 OUTPUT FILE                            TRIM TO VIDEO DURATION
    pass

In [14]:
#AddOrReplaceAudio("resources/video/sample1.mp4", "resources/audio/Amazon.mp3", "results/t3", "result")

## Task 3.4: Combine Videos

In [96]:
# Write your function here.
VIDEO_FORMATS = ["mp4", "avi"]
# Replace the parameters and return type of the following function according to the task specification.
def CombineVideos(
        inVideoLibFolder: str, \
        outFolder: str,         \
        outVideo: str) -> None:
    
    outputDir(outFolder)#Generate the output folder if it doesn't exist
    
    global VIDEO_FORMATS
    VIDEOS = [glob.glob(inVideoLibFolder+"/*."+n) for n in VIDEO_FORMATS]

    #Output video properties
    height = 0
    width = 0
    fps = 0
    
    #                GRAB ALL MP4 FILES                 GRAB ALL AVI FILES
    for v in VIDEOS:
        for video in v:
            md = VideoMetadataExtractor(*video.rsplit("/", 1))
            height = max(height, md["vHeight"])
            width = max(width, md["vWidth"])
            fps = max(fps, md["vFPS"])

    #List of all video inputs in ffmpeg 
    inputs = []

    for v in VIDEOS:
        for video in v:
            inp = ffmpeg.input(video)
            inputs += [
                inp.video
                    .filter('scale', f"{width}x{height}"),
                inp.audio] #Divide the video from the audio for the concat
    
    #https://www.reddit.com/r/learnpython/comments/rbr32l/combine_list_of_video_into_single_video/
    node = ffmpeg.concat(*inputs, v=1, a=1).node
    ffmpeg.output(node[0], node[1], outFolder+"/"+outVideo+".mp4", r=fps)\
    .run(overwrite_output=True)

# Replace the parameters and return type of the following function according to the task specification.
def VideoMetadataExtractor(
        videoFolder: str, \
        video: str =None) -> dict:
    
    global VIDEO_FORMATS
    
    #If no video is provided , a dataframe is generated
    if(video == None):
        files = []
    
        for format_videos in [glob.glob(videoFolder+"/*."+n) for n in VIDEO_FORMATS]: 
            for vid in format_videos: 
                files.append(vid)

        return pd.concat([pd.DataFrame(data=VideoMetadataExtractor(videoFolder, data.split("/")[-1]), index=[i]) for i, data in enumerate(files)])
    
    #If a video is provided, the information is returned in a dictionary format both to fill the dataframe or grab directly the information
    streams = ffmpeg.probe(videoFolder+"/"+video)['streams']
    videoStream = streams[0]
    audioStream = streams[1]
    
    return {
        "vCodec"      : videoStream["codec_name"],
        "vCodecID"    : int(videoStream["codec_tag"], 16),
        "vDur"        : videoStream["duration"],
        "vFPS"        : int(round(eval(videoStream["avg_frame_rate"]), 0)),
        "vHeight"     : videoStream["height"],
        "vWidth"      : videoStream["width"],
        
        "aCodec"      : audioStream["codec_name"],
        "aCodecID"    : int(audioStream["codec_tag"], 16),
        "aChannels"   : audioStream["channels"],
        "aSampleRate" : audioStream["sample_rate"],
        "aBitRate"    : audioStream["bit_rate"],
    }

In [98]:
# Test your function here.
#VideoMetadataExtractor("resources/video")
#CombineVideos("resources/video", "results/t4", "combination")

,vCodec,vCodecID,vDur,vFPS,vHeight,vWidth,aCodec,aCodecID,aChannels,aSampleRate,aBitRate
0,h264,828601953,42.375667,30,240,352,aac,1630826605,2,44100,127761
1,h264,828601953,7.107100,30,480,640,aac,1630826605,2,44100,128289
2,h264,875967048,7.173833,24,486,720,aac,255,2,44100,128000


## Task 3.5: Blend an Image in a Video

In [8]:
# Write your code here!

# Replace the parameters and return type of the following function according to the task specification.
def AddFadingImage(inVideo: str, inImg: str, time: float, outFolder: str, outVideo: str) -> None: 

    outputDir(outFolder)#Generate the output folder if it doesn't exist

    t = float(ffmpeg.probe(inVideo)["streams"][0]["duration"])#Total time

    duration = max(0.01, min((t - time)/2, 3)) #Fading time (the difference between the visible time and the total time bounded to [0.01, 3] seconds)

    img = ffmpeg.input(inImg, t=42, loop=1)\
                .filter("scale", "-1", "100")\
                .filter("fade", type="in", st=0, d=duration, alpha=1)\
                .filter("fade", t="out", st=min(duration+time, t - duration), d=duration, alpha=1)#The start time (st) is set to the fade in + duration or the total time - the fade duration if the input duration is too long
    
    
    ffmpeg.filter([ffmpeg.input(inVideo), img], 'overlay', 10, 10)\
        .output(outFolder + "/" + outVideo, map= "0:a")\
        .run(overwrite_output=True)

In [9]:
# Test your function here.
#AddFadingImage("resources/video/sample1.mp4", "resources/images/elephant14m.png", 430, "results/t5", "result.mp4")

Error: ffmpeg error (see stderr output for detail)

## Task 3.6: Create a Video Collage - blend two videos into one

In [14]:
# Write your function here.

#Better way to select layout style
class Layout(Enum):
    HORIZONTAL = True
    VERTICAL = False
# Replace the parameters and return type of the following function according to the task specification. 
def VideoClipMixer(
        inVideo1: str,       \
        inVideo2: str,        \
        layout: Layout, \
        outFolder: str,         \
        outVideo: str) -> None: 
    
    outputDir(outFolder)#Generate the output folder if it doesn't exist
    
    i = int(layout.value) #turn the Layout boolean into an int (HORIZONTAL = True = 1, VERTICAL = False = 0)
    ni = 1-i #negated i
    
    p1 = ffmpeg.probe(inVideo1)["streams"][0]
    p2 = ffmpeg.probe(inVideo2)["streams"][0]
    
    #get the video streams' dimensions
    p1 = [int(p1["width"]), int(p1["height"])]
    p2 = [int(p2["width"]), int(p2["height"])]
    
    #get the max of the two
    dimensions = [str(max(p1[0], p2[0])), str(max(p1[1], p2[1]))]
    
    dimensions[ni] = "temp" #only keep the shared axis (width in VERTICAL, height in HORIZONTAL)
    
    #Turn the shared dimension into a proportional scale
    p1[ni] *= int(dimensions[i]) / p1[i]
    p2[ni] *= int(dimensions[i]) / p2[i]
    
    #scale the videos in the shared axis
    in0 = ffmpeg.input(inVideo1).filter("scale","x".join(dimensions).replace("temp", str(int(p1[ni]))))
    in1 = ffmpeg.input(inVideo2).filter("scale","x".join(dimensions).replace("temp", str(int(p1[ni]))))
    
    #join the videos in the selected direction, vstack if VERTICAL, hstack if HORIZONTAL
    out = ffmpeg.filter((in0, in1), ["vstack", "hstack"][i])
    #                                                 FPS (if not set, infinite frames are generated)
    ffmpeg.output(out, outFolder+"/"+outVideo+".mp4", r=60)\
        .run(overwrite_output=True)
    


In [15]:
# Test your function here.
#VideoClipMixer("resources/video/sample2.mp4", "resources/video/sample3.avi", Layout.VERTICAL, "results/t6", "output")